# mdx VMのデプロイとVCP既存サーバ (SSH) モード セットアップ

このNotebookでは、mdx仮想マシンのデプロイと、VCノードとして利用するためのセットアップを行うための準備を行います。

## Notebook実行環境の準備

[mdx-rest-client-python](https://github.com/nii-gakunin-cloud/mdx-rest-client-python) のインストールと、VCノード用仮想マシンセットアップノートブックのダウンロードを行います。  
ダウンロードしたノートブックは、以下の規則に沿ったファイル名で現在のワークディレクトリに配置します。  
以下のセルを実行すると、ノートブックの作成とそのノートブックを開くためのリンク出力を行います。

```
{date}_{i}_{fid}_{file_name}

date: 実行日
i: 実行する度+1したものを付与（同日中に同じファイルを作成した場合のファイル名重複回避）
fid: 本ファイルのノートブック番号と同じものを付与
file_name: github上のファイル名
```


In [ ]:
import datetime
import os
from pathlib import Path
import requests
from urllib.parse import urlparse

from IPython.display import display, HTML

today = datetime.datetime.now().strftime('%Y%m%d')
fid = 941
file_url = 'https://raw.githubusercontent.com/nii-gakunin-cloud/mdx-rest-client-python/refs/heads/main/examples/deploy_vm_for_vcnode.ipynb'
file_name = Path(urlparse(file_url).path).name

for i in range(1, 100):
    fname = f'{today}_{i:02}_{fid}_{file_name}'
    fpath = os.path.join(Path().resolve(), fname)
    if not os.path.isfile(fname):
        response = requests.get(file_url)
        if response.status_code == 200:
            with open(fname, 'wb') as f:
                f.write(response.content)
            print(f'File created: {fname}')
        else:
            print(f'error{response.status_code}: {response.text}')
        break

display(HTML(f'<a href="{fname}" target="_blank">ノートブックを別タブで開く</a>'))